<a href="https://colab.research.google.com/github/trinade96/Fortiate_Project/blob/master/Fortiate_DST_5(all_txns_csv).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **DST 5(all_txns.csv)**
##### Trina De

In [0]:
!pip install folium==0.2.1

     |████████████████████████████████| 71kB 2.2MB/s 
  Created wheel for folium: filename=folium-0.2.1-cp36-none-any.whl size=79979 sha256=65fdebf4c32d42206f854d8ef33a00166ecc35692ca20b130d320839410df112
  Stored in directory: /root/.cache/pip/wheels/b8/09/f0/52d2ef419c2aaf4fb149f92a33e0008bdce7ae816f0dd8f0c5
Successfully built folium
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [0]:
!pip install coverage==3.7.1

In [0]:
!pip install pycm

     |████████████████████████████████| 61kB 2.0MB/s 
     |████████████████████████████████| 522kB 7.1MB/s 
     |████████████████████████████████| 235kB 24.1MB/s 
ERROR: datascience 0.10.6 has requirement coverage==3.7.1, but you'll have coverage 5.0.1 which is incompatible.
ERROR: coveralls 0.5 has requirement coverage<3.999,>=3.6, but you'll have coverage 5.0.1 which is incompatible.
  Found existing installation: coverage 3.7.1
    Uninstalling coverage-3.7.1:
      Successfully uninstalled coverage-3.7.1


In [0]:
!pip install scikit-tda

     |████████████████████████████████| 81kB 2.4MB/s 
     |████████████████████████████████| 102kB 4.4MB/s 
  Created wheel for ripser: filename=ripser-0.4.1-cp36-cp36m-linux_x86_64.whl size=423313 sha256=5d345adb97f42ce05131d29ff3757e41dc01c91911dac8a23bfc49663d47e732
  Stored in directory: /root/.cache/pip/wheels/d4/12/da/56d55c3af62ebd5e0684d521f3e58c1a85ac312502c9e2d47d
  Created wheel for hopcroftkarp: filename=hopcroftkarp-1.2.5-py2.py3-none-any.whl size=18091 sha256=3882fef7b269a641d63c7d6bba48735db1a7d497fae77928acd6aa2527ad949a
  Stored in directory: /root/.cache/pip/wheels/2b/e1/c9/1993c7f7f114b7d3fb2d3e895e02157a7ebf554861e9e54e01
Successfully built ripser hopcroftkarp


In [0]:
!pip install kmapper

##### **Packages Used**

In [0]:
import numpy as np
import pandas as pd
import time
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from scipy.stats import spearmanr
import pycm
from pycm import *
import kmapper as km
from sklearn.cluster import DBSCAN
import sklearn
from sklearn import ensemble
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

##### **Loading Notebook from Google Drive**

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
!cd /gdrive/My\ Drive/

In [0]:
missing_values = ["n/a", "na", "--", ' ', "NAN","nan","NA","NaN"]
data = pd.read_csv("/gdrive/My Drive/all_txns.csv",na_values=missing_values)
data.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,7,22,25,27,33,39,50,51,53,59,62,72,78,93) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,...,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120
0,1,1100,500360265637,265637,20-JUL-18 11.34.26,20-JUL-18 11.34.26,0,0,20-JUL-18 11.34.27,3.871512e+14,NaN,?,?,?,29-MAR-16 09.02.47,01-MAR-20 00.00.00,704684,367123,0,40,0.0,1,0,0,9,4,0,2,0.0,0.0,0.0,1.0,0,???,220.00,0.0,356,0.0,220.00,356,...,NaN,NaN,NaN,AMI,ACS,784.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0,NaN,NaN,AXASK05AEVV,5.0,NaN,NaN,NaN,485855.29,NaN,NaN,NaN,0,0,NaN,NaN,NaN,???,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1100,500360403436,403436,20-JUL-18 11.35.39,20-JUL-18 11.35.39,0,0,20-JUL-18 11.35.39,3.871513e+14,NaN,?,?,?,03-SEP-16 19.58.40,01-SEP-20 00.00.00,704684,367123,0,40,0.0,1,0,0,9,1,0,2,0.0,0.0,0.0,1.0,0,???,42.00,0.0,356,0.0,42.00,356,...,NaN,NaN,NaN,AMI,ACS,26.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0,NaN,NaN,AXASK05AEVV,5.0,NaN,NaN,NaN,117389.06,NaN,NaN,NaN,0,0,NaN,NaN,NaN,???,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1100,500360483836,483836,20-JUL-18 11.35.48,20-JUL-18 11.35.48,0,0,20-JUL-18 11.35.48,3.871510e+14,NaN,?,?,?,02-DEC-17 19.48.22,01-DEC-22 00.00.00,704684,367123,0,40,0.0,1,0,0,9,4,0,2,0.0,0.0,0.0,1.0,0,???,108.32,0.0,356,0.0,108.32,356,...,NaN,NaN,NaN,AMI,ACS,917.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0,NaN,NaN,AXASK05AEVV,5.0,NaN,NaN,NaN,102000.95,NaN,NaN,NaN,0,0,NaN,NaN,NaN,???,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1100,500360094946,94946,20-JUL-18 11.36.09,20-JUL-18 11.36.09,0,3,20-JUL-18 11.36.09,3.871510e+14,NaN,?,?,?,23-JAN-18 19.34.41,01-JAN-23 00.00.00,704684,367123,0,40,0.0,1,0,0,9,4,0,2,0.0,0.0,0.0,1.0,0,???,173.04,0.0,356,0.0,173.04,356,...,NaN,NaN,NaN,AMI,ACS,917.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0,NaN,NaN,AXASK05AEVV,5.0,NaN,NaN,NaN,59676.28,NaN,NaN,NaN,0,0,NaN,NaN,NaN,???,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1100,500360135131,135131,20-JUL-18 11.36.13,20-JUL-18 11.36.13,0,0,20-JUL-18 11.36.13,3.871518e+14,NaN,?,?,?,26-APR-18 19.52.38,01-APR-23 00.00.00,704684,367123,0,40,0.0,1,0,0,9,4,0,2,0.0,0.0,0.0,1.0,0,???,241.00,0.0,356,0.0,241.00,356,...,NaN,NaN,NaN,AMI,ACS,917.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0,NaN,NaN,AXASK05AEVV,5.0,NaN,NaN,NaN,50250.00,NaN,NaN,NaN,0,0,NaN,NaN,NaN,???,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### **Removing completely NaN and constant columns**


In [0]:
indices1 = []
indices2 = []
start = time.time()
for col in data.columns:
  if data[col].isnull().sum() == len(data[col]):
    indices1.append(col)
  elif len(set(data[col])) == 1:
    indices2.append(col)
data=data.drop(indices1 + indices2,1)
data.shape
end = time.time()
print("Time taken to drop NaN and constant columns are:{}".format(end-start))

Time taken to drop NaN and constant columns are:65.66611051559448


##### It takes roughly about a minute to drop all the NaN and constant columns.

##### **Put column data type identifiers everywhere and for the NaNs we will convert them to a string or fixed number so that we can identify that as a category or a missing value later**

In [0]:
# Function that only checks categorical
def check_categorical(l):
   if(len(set(l))==(len(l))):
     return False
   elif(len(set(l))==1):
     return False
   else :
     return True

In [0]:
## It treats every missing value differently
## And so far we have come across 3 things that a column omitting its NaN values might be;
## Of dtype 'int64', 'float' or 'object'(this includes date)
## Of dtype dtype 'O' which means python object one example of which are values such as True or False
## Now depending on the data type of the rest of the column we will modify the NaN values 
# data['V76'][pd.isna(data['V76'])] = 0
# data['V76']

In [0]:
# set(data['V76'])

In [0]:
column_identifiers = {}
start = time.time()
for col in data.columns:
  l = data[col][pd.notna(data[col])]
  #now we will start working with l
  
  if(l.dtype == 'O' ):
    data[col][pd.isna(data[col])] = 'unknown_Python_object'
    column_identifiers[col] = 'Categorical'
  elif(l.dtype == 'object'):
    data[col][pd.isna(data[col])] = 'unknown_object_type'
    column_identifiers[col] = 'Categorical'
  elif(l.dtype == 'int64'):
    if(check_categorical(l) == True):
      data[col][pd.isna(data[col])] = 'unknown_numeric_category'
      column_identifiers[col] = 'Categorical'
    else:
      data[col][pd.isna(data[col])] = -1
      column_identifiers[col] = 'Integer'
  elif(l.dtype == 'float'):
    if(check_categorical(l) == True):
      data[col][pd.isna(data[col])] = 'unknown_float_category'
      column_identifiers[col] = 'Categorical'
    else:
      data[col][pd.isna(data[col])] = -1.0
      column_identifiers[col] = 'Float'
  elif(l.dtype == 'str'):
      data[col][pd.isna(data[col])] = -'unknown_string'
      column_identifiers[col] = 'String'
end = time.time()
print("Time taken to assign column datatypes:{}".format(end-start))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: Setting

Time taken to assign column datatypes:1.7266528606414795


In [0]:
column_identifiers

{'V1': 'Integer',
 'V10': 'Categorical',
 'V102': 'Categorical',
 'V113': 'Categorical',
 'V114': 'Categorical',
 'V12': 'Categorical',
 'V13': 'Categorical',
 'V14': 'Categorical',
 'V15': 'Categorical',
 'V16': 'Categorical',
 'V18': 'Categorical',
 'V19': 'Categorical',
 'V2': 'Categorical',
 'V20': 'Categorical',
 'V21': 'Float',
 'V22': 'Categorical',
 'V23': 'Categorical',
 'V24': 'Categorical',
 'V25': 'Categorical',
 'V26': 'Categorical',
 'V27': 'Categorical',
 'V28': 'Categorical',
 'V29': 'Categorical',
 'V3': 'Categorical',
 'V30': 'Categorical',
 'V31': 'Categorical',
 'V32': 'Categorical',
 'V33': 'Categorical',
 'V34': 'Categorical',
 'V35': 'Categorical',
 'V36': 'Float',
 'V37': 'Categorical',
 'V38': 'Categorical',
 'V39': 'Categorical',
 'V4': 'Categorical',
 'V40': 'Categorical',
 'V41': 'Categorical',
 'V43': 'Categorical',
 'V44': 'Categorical',
 'V48': 'Float',
 'V49': 'Categorical',
 'V5': 'Categorical',
 'V50': 'Categorical',
 'V51': 'Categorical',
 'V52': 'Cat

##### **Seperating out the columns as type 'Float', 'Integer', 'Categorical'**

In [0]:
int_list = []
flo_list = []
cat_list = []
for key in column_identifiers.keys():
  if column_identifiers[key] == 'Categorical':
    cat_list.append(key)
  elif column_identifiers[key] == 'Float':
    flo_list.append(key)
  elif column_identifiers[key] == 'Integer':
    int_list.append(key)
print(int_list)
print(flo_list)
print(cat_list)


['V1']
['V21', 'V36', 'V48', 'V90']
['V2', 'V3', 'V4', 'V5', 'V6', 'V8', 'V9', 'V10', 'V12', 'V13', 'V14', 'V15', 'V16', 'V18', 'V19', 'V20', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V37', 'V38', 'V39', 'V40', 'V41', 'V43', 'V44', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V66', 'V67', 'V68', 'V69', 'V70', 'V73', 'V76', 'V77', 'V79', 'V84', 'V85', 'V86', 'V89', 'V94', 'V97', 'V98', 'V102', 'V113', 'V114']


In [0]:
data.head()

,V1,V2,V3,V4,V5,V6,V8,V9,V10,V12,V13,V14,V15,V16,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V43,V44,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V66,V67,V68,V69,V70,V73,V76,V77,V79,V84,V85,V86,V89,V90,V94,V97,V98,V102,V113,V114
0,1,1100,500360265637,265637,20-JUL-18 11.34.26,20-JUL-18 11.34.26,0,20-JUL-18 11.34.27,3.871512e+14,?,?,?,29-MAR-16 09.02.47,01-MAR-20 00.00.00,367123,0,40,0.0,1,0,0,9,4,0,2,0,0,0,1,0,???,220.00,0.0,356,0.0,220.00,356,0.0,215.89,356,0.0,356,123456,777777,356,123456,777777,4722,6R00G052,9825368954,PAY U OLA CABS PG \\\122001\\,6,200267,0,4,2,4,1,0,1,1,1001,unknown_float_category,2,6078503409872,AMI,ACS,784,0,0.0,0,AXASK05AEVV,5,485855,unknown_Python_object,unknown_Python_object
1,2,1100,500360403436,403436,20-JUL-18 11.35.39,20-JUL-18 11.35.39,0,20-JUL-18 11.35.39,3.871513e+14,?,?,?,03-SEP-16 19.58.40,01-SEP-20 00.00.00,367123,0,40,0.0,1,0,0,9,1,0,2,0,0,0,1,0,???,42.00,0.0,356,0.0,42.00,356,0.0,41.21,356,0.0,356,123456,777777,356,123456,777777,4722,6R00G052,9825393929,PAYTM.COM TRAVEL PG \\NOIDA\201310\\,6,200340,0,4,2,4,1,0,1,1,1001,unknown_float_category,2,6051404919372,AMI,ACS,26,0,0.0,0,AXASK05AEVV,5,117389,unknown_Python_object,unknown_Python_object
2,3,1100,500360483836,483836,20-JUL-18 11.35.48,20-JUL-18 11.35.48,0,20-JUL-18 11.35.48,3.871510e+14,?,?,?,02-DEC-17 19.48.22,01-DEC-22 00.00.00,367123,0,40,0.0,1,0,0,9,4,0,2,0,0,0,1,0,???,108.32,0.0,356,0.0,108.32,356,0.0,106.40,356,0.0,356,123456,777777,356,123456,777777,7399,6R00G052,9826826968,BOOK MY SHOW\\\400049\\,6,200348,0,4,2,4,1,0,1,1,1001,unknown_float_category,2,6039443564572,AMI,ACS,917,0,0.0,0,AXASK05AEVV,5,102001,unknown_Python_object,unknown_Python_object
3,4,1100,500360094946,94946,20-JUL-18 11.36.09,20-JUL-18 11.36.09,3,20-JUL-18 11.36.09,3.871510e+14,?,?,?,23-JAN-18 19.34.41,01-JAN-23 00.00.00,367123,0,40,0.0,1,0,0,9,4,0,2,0,0,0,1,0,???,173.04,0.0,356,0.0,173.04,356,0.0,169.98,356,0.0,356,123456,777777,356,123456,777777,7399,6R00G052,9826826968,BOOK MY SHOW\\\400049\\,6,500370,0,4,2,4,1,0,1,1,1001,unknown_float_category,2,6039440900571,AMI,ACS,917,0,0.0,0,AXASK05AEVV,5,59676.3,unknown_Python_object,unknown_Python_object
4,5,1100,500360135131,135131,20-JUL-18 11.36.13,20-JUL-18 11.36.13,0,20-JUL-18 11.36.13,3.871518e+14,?,?,?,26-APR-18 19.52.38,01-APR-23 00.00.00,367123,0,40,0.0,1,0,0,9,4,0,2,0,0,0,1,0,???,241.00,0.0,356,0.0,241.00,356,0.0,236.73,356,0.0,356,123456,777777,356,123456,777777,7399,6R00G052,9826826968,BOOK MY SHOW\\\400049\\,6,400374,0,4,2,4,1,0,1,1,1001,unknown_float_category,2,6039450425576,AMI,ACS,917,0,0.0,0,AXASK05AEVV,5,50250,unknown_Python_object,unknown_Python_object


##### We will drop columns 'V5' , 'V6', 'V9',  'V15', 'V16' from the categorical list because those are date columns which we will deal with differently.


In [0]:
remove = ['V5','V6','V9','V15','V16']
for i in remove:
  cat_list = list(filter(lambda a: a != i, cat_list))
cat_list

['V2',
 'V3',
 'V4',
 'V8',
 'V10',
 'V12',
 'V13',
 'V14',
 'V18',
 'V19',
 'V20',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'V29',
 'V30',
 'V31',
 'V32',
 'V33',
 'V34',
 'V35',
 'V37',
 'V38',
 'V39',
 'V40',
 'V41',
 'V43',
 'V44',
 'V49',
 'V50',
 'V51',
 'V52',
 'V53',
 'V54',
 'V55',
 'V56',
 'V57',
 'V58',
 'V59',
 'V60',
 'V61',
 'V62',
 'V63',
 'V66',
 'V67',
 'V68',
 'V69',
 'V70',
 'V73',
 'V76',
 'V77',
 'V79',
 'V84',
 'V85',
 'V86',
 'V89',
 'V94',
 'V97',
 'V98',
 'V102',
 'V113',
 'V114']

##### We will now proceed to model fitting for the different combinations:
> Categorical vs. Continuous

> Categorical vs. Categorical

> Categorical vs. Categorical vs. Categorical (3 Categorical variables)

> Comparing date type columns(Also under Categorical right now)



##### **Continuous vs.  Categorical**
##### We will use two approaches here:
> Simple range method

>Fitting distributions(Gaussian)


##### **Simple range method**

In [0]:
## Function to output expected continuous amount given a categorical and a float column
def cat_vs_flo(c,f):
  if column_identifiers[c] != 'Categorical':
    print ('Check Data type of 1st argument')
  elif column_identifiers[f] != 'Float':
    print('Check Data type of 2nd argument')
  else:
    df_temp = data[[c,f]]
    grouped_data = pd.DataFrame(df_temp[c].groupby([df_temp[f]]).mean())
    return grouped_data

In [0]:
## Function to output expected integer values given a categorical and an integer column
def cat_vs_int(c,i):
  if column_identifiers[c] != 'Categorical':
    print ('Check Data type of 1st argument')
  elif column_identifiers[f] != 'Integer':
    print('Check Data type of 2nd argument')
  else:
    df_temp = data[[c,i]]
    grouped_data = pd.DataFrame(df_temp[c].groupby([df_temp[i]]).mean())
    return grouped_data


In [0]:
cat_vs_flo(cat_list[0],flo_list[0])

,V2
V21,
-1.0,1200.000000
0.0,1196.162239


##### So given the catagorical variable value our estimate of the continuous variable value is the mean of previous values seen for that category

In [0]:
cat_vs_int(cat_list[0],int_list[0])

,V2
V1,
1,1100
2,1100
3,1100
4,1100
5,1100
...,...
65533,1200
65534,1200
65535,1200


In [0]:
## Function to output contingency table given two categorical columns
def cat_vs_cat(c1,c2):
  if column_identifiers[c1] != 'Categorical':
    print ('Check Data type of 1st argument')
  elif column_identifiers[c2] != 'Categorical':
    print('Check Data type of 2nd argument')
  else:
    df_temp = data[[c1,c2]]
    grouped_data = pd.crosstab(df_temp[c1],df_temp[c2], dropna = False)
    return grouped_data

In [0]:
cat_vs_cat(cat_list[0],cat_list[1])

V3,820103380808,820103451495,820103501965,820103502703,820108174300,820200001368,820200025788,820200059368,820200112893,820200115312,820200128704,820200131388,820200225707,820200705087,820200713812,820200893379,820200905514,820200930241,820200948460,820200985197,820201185550,820201201015,820201289305,820201293163,820201705664,820203001479,820203001940,820203002636,820203002654,820203002686,820203002901,820203002991,820203004665,820203005006,820203005083,820203005089,820203005230,820203005847,820203022270,820203026897,...,820264803783,820264806063,820266381954,820268364534,820271035128,820271694601,820271700365,820271716478,820271716609,820271747378,820271749199,820271797785,820272467075,820272470228,820272478547,820272479082,820272491517,820272494261,820272495873,820272509520,820272521794,820272526345,820273413410,820273414721,820273420144,820273435602,820273456684,820273462223,820273464442,820273471179,820273482007,820273489137,820273661869,820274629645,820274645972,820274648826,820274652123,820274652778,820277285664,????????????
V2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,882
1200,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,30


##### So given a value which is one of the row labels we can traverse that row for the maximum value and return the column label corresponding to it.

In [0]:
## Function to output contingency table given 3 categorical columns
def cat_vs_cat_vs_cat(c1,c2,c3):
  if column_identifiers[c1] != 'Categorical':
    print ('Check Data type of 1st argument')
  elif column_identifiers[c2] != 'Categorical':
    print('Check Data type of 2nd argument')
  elif column_identifiers[c3] != 'Categorical':
    print('Check Data type of 3rd argument')
  else:
    df_temp = data[[c1,c2,c3]]
    grouped_data = pd.crosstab(df_temp[c1],[df_temp[c2],df_temp[c3]], dropna = False)
    return grouped_data

##### We should now divide up the categorical into LN(Long Nominal) and SN(Short Nominal)

In [0]:
categorical_type = {}
for c in cat_list:
  if(len(set(data[c]))>40):
    categorical_type[c] = 'LN'
  else:
    categorical_type[c] = 'SN'
print(categorical_type)

{'V2': 'SN', 'V3': 'LN', 'V4': 'LN', 'V8': 'SN', 'V10': 'LN', 'V12': 'SN', 'V13': 'SN', 'V14': 'SN', 'V18': 'SN', 'V19': 'SN', 'V20': 'SN', 'V22': 'SN', 'V23': 'SN', 'V24': 'SN', 'V25': 'SN', 'V26': 'SN', 'V27': 'SN', 'V28': 'SN', 'V29': 'SN', 'V30': 'SN', 'V31': 'SN', 'V32': 'SN', 'V33': 'SN', 'V34': 'SN', 'V35': 'LN', 'V37': 'LN', 'V38': 'SN', 'V39': 'LN', 'V40': 'SN', 'V41': 'LN', 'V43': 'LN', 'V44': 'SN', 'V49': 'LN', 'V50': 'LN', 'V51': 'SN', 'V52': 'LN', 'V53': 'LN', 'V54': 'SN', 'V55': 'LN', 'V56': 'LN', 'V57': 'LN', 'V58': 'LN', 'V59': 'SN', 'V60': 'LN', 'V61': 'SN', 'V62': 'SN', 'V63': 'SN', 'V66': 'SN', 'V67': 'SN', 'V68': 'SN', 'V69': 'SN', 'V70': 'SN', 'V73': 'SN', 'V76': 'SN', 'V77': 'SN', 'V79': 'LN', 'V84': 'SN', 'V85': 'SN', 'V86': 'LN', 'V89': 'SN', 'V94': 'SN', 'V97': 'LN', 'V98': 'SN', 'V102': 'LN', 'V113': 'SN', 'V114': 'SN'}


In [0]:
len(set(data[cat_list[1]]))

64540

##### We see that our function identifies it correctly as Long Nominal LN.

In [0]:
cat_vs_cat_vs_cat(cat_list[0],cat_list[3],cat_list[5])

V8      1                0              1        ...  H      I         J       
V12     1      2      ?  1    2     ?   1     2  ...  2   ?  1  2   ?  1  2   ?
V2                                               ...                           
1100    0      0      0  0  437  1186   0     1  ...  0  20  0  0  10  0  0  15
1200  161  34897  22284  0    0     0  12  2437  ...  0   0  0  0   0  0  0   0

[2 rows x 54 columns]

In [0]:
# for i in cat_list:
#   for j in flot_list:
#     temp = data[[i,j]]
#     #grouped_temp = temp.groupby(by=j)
#     grouped_data = pd.DataFrame(temp[i].groupby([temp[j]]).mean())
#     for row in grouped_data.iterrows

    

In [0]:
# change question mark to unknown category(already taken care of in categorical encoding)
# category vs category modelling ?(take the most probable given the category)
# category vs continuous (taken the mean over the category)
# Date type columns compare (date type and numeic type)
# DE 18, 22, 49 ()
# Target Data (don't know which column that is over here)


##### **ROUGH WORK**

In [0]:

data2['TARGET'] = [x if x == 'Fraud' else 'Not Fraud' for x in list(data2['TARGET'])]
set(data2['TARGET'])

In [0]:
set(data['V120'].values)

In [0]:
data.columns

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V8', 'V9', 'V10', 'V12', 'V13',
       'V14', 'V15', 'V16', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24',
       'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34',
       'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V43', 'V44', 'V48',
       'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58',
       'V59', 'V60', 'V61', 'V62', 'V63', 'V66', 'V67', 'V68', 'V69', 'V70',
       'V73', 'V76', 'V77', 'V79', 'V84', 'V85', 'V86', 'V89', 'V90', 'V94',
       'V97', 'V98', 'V102', 'V113', 'V114'],
      dtype='object')

In [0]:
data['V113'].dtype.name

'object'

In [0]:
set(data['V14'])

{'1', '6', '?'}

In [0]:
data.replace("nan",0)
data.head()

,V1,V2,V3,V4,V5,V6,V8,V9,V10,V12,V13,V14,V15,V16,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V43,V44,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V66,V67,V68,V69,V70,V73,V76,V77,V79,V84,V85,V86,V89,V90,V94,V97,V98,V102,V113,V114
0,1,1100,500360265637,265637,20-JUL-18 11.34.26,20-JUL-18 11.34.26,0,20-JUL-18 11.34.27,3.871512e+14,?,?,?,29-MAR-16 09.02.47,01-MAR-20 00.00.00,367123,0,40,0.0,1,0,0,9,4,0,2,0.0,0.0,0.0,1.0,0,???,220.00,0.0,356,0.0,220.00,356,0.0,215.89,356,0.0,356,123456,777777,356,123456.0,777777,4722,6R00G052,9825368954,PAY U OLA CABS PG \\\122001\\,6,200267,0,4,2,4,1,0,1,1,1001,NaN,2.0,6078503409872,AMI,ACS,784.0,0.0,0.0,0,AXASK05AEVV,5.0,485855.29,NaN,NaN
1,2,1100,500360403436,403436,20-JUL-18 11.35.39,20-JUL-18 11.35.39,0,20-JUL-18 11.35.39,3.871513e+14,?,?,?,03-SEP-16 19.58.40,01-SEP-20 00.00.00,367123,0,40,0.0,1,0,0,9,1,0,2,0.0,0.0,0.0,1.0,0,???,42.00,0.0,356,0.0,42.00,356,0.0,41.21,356,0.0,356,123456,777777,356,123456.0,777777,4722,6R00G052,9825393929,PAYTM.COM TRAVEL PG \\NOIDA\201310\\,6,200340,0,4,2,4,1,0,1,1,1001,NaN,2.0,6051404919372,AMI,ACS,26.0,0.0,0.0,0,AXASK05AEVV,5.0,117389.06,NaN,NaN
2,3,1100,500360483836,483836,20-JUL-18 11.35.48,20-JUL-18 11.35.48,0,20-JUL-18 11.35.48,3.871510e+14,?,?,?,02-DEC-17 19.48.22,01-DEC-22 00.00.00,367123,0,40,0.0,1,0,0,9,4,0,2,0.0,0.0,0.0,1.0,0,???,108.32,0.0,356,0.0,108.32,356,0.0,106.40,356,0.0,356,123456,777777,356,123456.0,777777,7399,6R00G052,9826826968,BOOK MY SHOW\\\400049\\,6,200348,0,4,2,4,1,0,1,1,1001,NaN,2.0,6039443564572,AMI,ACS,917.0,0.0,0.0,0,AXASK05AEVV,5.0,102000.95,NaN,NaN
3,4,1100,500360094946,94946,20-JUL-18 11.36.09,20-JUL-18 11.36.09,3,20-JUL-18 11.36.09,3.871510e+14,?,?,?,23-JAN-18 19.34.41,01-JAN-23 00.00.00,367123,0,40,0.0,1,0,0,9,4,0,2,0.0,0.0,0.0,1.0,0,???,173.04,0.0,356,0.0,173.04,356,0.0,169.98,356,0.0,356,123456,777777,356,123456.0,777777,7399,6R00G052,9826826968,BOOK MY SHOW\\\400049\\,6,500370,0,4,2,4,1,0,1,1,1001,NaN,2.0,6039440900571,AMI,ACS,917.0,0.0,0.0,0,AXASK05AEVV,5.0,59676.28,NaN,NaN
4,5,1100,500360135131,135131,20-JUL-18 11.36.13,20-JUL-18 11.36.13,0,20-JUL-18 11.36.13,3.871518e+14,?,?,?,26-APR-18 19.52.38,01-APR-23 00.00.00,367123,0,40,0.0,1,0,0,9,4,0,2,0.0,0.0,0.0,1.0,0,???,241.00,0.0,356,0.0,241.00,356,0.0,236.73,356,0.0,356,123456,777777,356,123456.0,777777,7399,6R00G052,9826826968,BOOK MY SHOW\\\400049\\,6,400374,0,4,2,4,1,0,1,1,1001,NaN,2.0,6039450425576,AMI,ACS,917.0,0.0,0.0,0,AXASK05AEVV,5.0,50250.00,NaN,NaN


In [0]:
data['V113'].dtype

dtype('O')

In [0]:
data['V113'].dtype == 'O'

True

In [0]:
set(data['V113'][pd.notna(data['V113'])])

{True}

In [0]:
data[data['V113'] == True]

,V1,V2,V3,V4,V5,V6,V8,V9,V10,V12,V13,V14,V15,V16,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V43,V44,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V66,V67,V68,V69,V70,V73,V76,V77,V79,V84,V85,V86,V89,V90,V94,V97,V98,V102,V113,V114
2892,2893,1200,820118387394,387394,21-JUL-18 00.00.01,20-JUL-18 18.30.01,1,20-JUL-18 18.30.01,4.157521e+15,?,?,?,19-OCT-15 18.43.07,01-OCT-20 00.00.00,404724,0,0,NaN,1,0,S,0,2,0,1,0.0,0.0,0.0,0.0,0,19900,476.71,0.0,50,0.0,405.90,356,78514611.0,406.03,356,0.0,528,479631,555555,528,479631.0,555555,4121,99999999,479631000200010,UBER TRIP RJX7B HELP.UBERhelp.uber.comNL,6,200001,0,4,0,1,1,0,1,1,1001,2.0,2.0,388201666011404,BS1,POS,28.0,0.0,0.0,0,NaN,7.0,100013.04,True,True
3206,3207,1200,820118140073,140073,21-JUL-18 00.15.45,20-JUL-18 18.45.45,1,20-JUL-18 18.45.45,4.281240e+15,?,?,?,16-OCT-14 13.54.36,01-OCT-19 00.00.00,417096,0,0,0.0,1,0,S,0,2,0,1,0.0,0.0,0.0,0.0,0,19900,13.33,0.0,826,0.0,1247.48,356,59358440.0,1253.70,356,0.0,528,498750,555555,528,498750.0,555555,4121,99999999,498750000083874,UBER TRIP MANGS HELP.UBERhelp.uber.comGB,6,800945,0,4,0,1,1,0,1,1,1001,2.0,2.0,388201675456231,BS1,POS,1135.0,0.0,0.0,0,NaN,7.0,339146.10,True,True
3278,3279,1200,820118839331,839331,21-JUL-18 00.21.19,20-JUL-18 18.51.19,1,20-JUL-18 18.51.20,4.422859e+15,?,?,?,08-SEP-17 19.41.02,01-SEP-22 00.00.00,431257,0,0,0.0,1,0,S,0,2,0,1,0.0,0.0,0.0,0.0,0,19900,3.28,0.0,840,0.0,234.59,356,57152134.0,234.67,356,0.0,826,454706,555555,826,454706.0,555555,5964,WPGTID01,28488525,WWW.ALIEXPRESS.COM LONDON GB,6,801280,0,4,0,1,1,0,1,1,1001,2.0,2.0,468201678794434,BS1,POS,796.0,0.0,0.0,0,NaN,7.0,47612.99,True,True
3325,3326,1200,820118096413,96413,21-JUL-18 00.24.13,20-JUL-18 18.54.13,1,20-JUL-18 18.54.13,4.422850e+15,?,?,?,07-APR-18 19.30.22,01-APR-23 00.00.00,431257,0,0,0.0,1,0,S,0,2,0,1,0.0,0.0,0.0,0.0,0,19900,21.29,0.0,840,0.0,1522.70,356,57152184.0,1523.20,356,0.0,826,454706,555555,826,454706.0,555555,5964,WPGTID01,28488525,WWW.ALIEXPRESS.COM LONDON GB,6,801453,0,4,0,1,1,0,1,1,1001,2.0,2.0,588201680535900,BS1,POS,796.0,0.0,0.0,0,NaN,7.0,44036.98,True,True
3358,3359,1200,820200364215,364215,21-JUL-18 00.26.24,21-JUL-18 00.26.24,1,21-JUL-18 00.26.24,4.422858e+15,?,?,?,28-AUG-15 20.40.12,01-AUG-20 00.00.00,431257,0,0,0.0,1,0,S,0,2,0,1,0.0,0.0,0.0,0.0,0,19900,3870.50,0.0,356,0.0,3870.50,356,61000000.0,3870.50,356,0.0,356,411075,555555,356,411075.0,555555,4112,38R00590,38RR00000000590,"RAZ*IRCTC, Statesman Ho New Delhi IN",6,201584,0,4,0,1,1,0,1,1,1001,2.0,2.0,468201681845992,BS1,POS,796.0,0.0,0.0,0,000002007648094224748219324809110E8B3CD5,5.0,42339.90,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63388,63392,1200,820216235060,235060,21-JUL-18 21.43.13,21-JUL-18 16.13.13,1,21-JUL-18 16.13.13,4.422858e+15,?,?,?,12-AUG-13 21.27.28,01-AUG-18 00.00.00,431257,0,0,0.0,1,0,S,0,2,0,1,0.0,0.0,0.0,0.0,0,19900,10.76,0.0,826,0.0,1011.99,356,59405112.0,1011.99,356,0.0,528,498750,555555,528,498750.0,555555,4121,99999999,498750000083874,UBER TRIP DC5TT HELP.UBERhelp.uber.comGB,6,978193,0,4,0,1,1,0,1,1,1001,2.0,2.0,308202583934254,BS1,POS,796.0,0.0,0.0,0,NaN,7.0,289082.15,True,True
63935,63939,1200,820216405130,405130,21-JUL-18 21.50.47,21-JUL-18 16.20.47,1,21-JUL-18 16.20.47,4.281240e+15,?,?,?,04-JUL-17 20.19.43,01-JUL-22 00.00.00,417096,0,0,0.0,1,0,S,0,2,0,1,0.0,0.0,0.0,0.0,0,19900,14.25,0.0,826,0.0,1340.23,356,59405123.0,1340.23,356,0.0,528,498750,555555,528,498750.0,555555,4121,99999999,498750000083874,UBER TRIP ICTUL HELP.UBERhelp.uber.comGB,6,478647,0,4,0,1,1,0,1,1,1001,2.0,2.0,468202588474887,BS1,POS,796.0,0.0,0.0,0,NaN,7.0,631246.92,True,True
64867,64871,1200,820216403233,403233,21-JUL-18 22.05.34,21-JUL-18 

In [0]:
from pandas.api.types import is_datetime64_any_dtype as is_datetime

df[[column for column in df.columns if is_datetime(df[column])]]

dtype('O')

In [0]:
data[['V2','V3']]

,V2,V3
0,1100,500360265637
1,1100,500360403436
2,1100,500360483836
3,1100,500360094946
4,1100,500360135131
...,...,...
65529,1200,820216192640
65530,1200,820216192618
65531,1200,820216768854
65532,1200,820216192746


In [0]:
type(data['V2'].groupby([data['V3']]).mean())

pandas.core.series.Series